<a href="https://colab.research.google.com/github/ZaidRZ/colab/blob/master/ZA_FAS_MNS_EWC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install avalanche-lib==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 4.3 MB/s 
     |████████████████████████████████| 1.9 MB 48.9 MB/s 
     |████████████████████████████████| 532 kB 67.3 MB/s 
     |████████████████████████████████| 427 kB 53.6 MB/s 
     |████████████████████████████████| 529 kB 69.8 MB/s 
     |████████████████████████████████| 182 kB 13.1 MB/s 
     |████████████████████████████████| 162 kB 58.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 40.5 MB/s 
     |████████████████████████████████| 158 kB 59.0 MB/s 
     |████████████████████████████████| 157 kB 66.6 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 157 kB 66.2 MB/s 
     |████████████████████████████████| 157 kB 60.7 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████

In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import EWC

scenario = SplitMNIST(n_experiences=5)
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

In [ ]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

In [ ]:
# log to Tensorboard
tb_logger = TensorboardLogger()

In [ ]:
# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

In [ ]:
# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

In [ ]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy=EWC(model, SGD(model.parameters(), lr=0.001, momentum=0.9),CrossEntropyLoss(), ewc_lambda = 1,
                train_mb_size=500, train_epochs=1, eval_mb_size=100, evaluator=eval_plugin)

In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))